# Projeto 2 - Ciência dos Dados

Nome: Antônio Fonseca

Nome: José Bechara

Serão permitidos grupos de três pessoas, mas com uma rubrica mais exigente. Grupos deste tamanho precisarão fazer um questionário de avaliação de trabalho em equipe

___
# Classificador automático de sentimento


## Preparando o ambiente no jupyter:

In [1]:
%%capture

#Instalando o tweepy
!pip install tweepy

In [2]:
import tweepy
import math
import os.path
import pandas as pd
import json
from random import shuffle
import emoji
import functools
import operator
import re
from time import sleep

___
## Autenticando no  Twitter

* Conta: ***@JosAntnioBecha2***

In [3]:
# #Dados de autenticação do twitter:

# #Coloque aqui o identificador da conta no twitter: @JosAntnioBecha2 

#leitura do arquivo no formato JSON
with open('auth.pass') as fp:    
    data = json.load(fp)

#Configurando a biblioteca. Não modificar
auth = tweepy.OAuthHandler(data['consumer_key'], data['consumer_secret'])
auth.set_access_token(data['access_token'], data['access_token_secret'])

___
## Etapas do projeto:

### Escolha de um produto e coleta das mensagens


In [4]:
#Produto escolhido:
produto = 'tesla'

#Quantidade mínima de mensagens capturadas:
n = 500
#Quantidade mínima de mensagens para a base de treinamento:
t = 300

#Filtro de língua, escolha uma na tabela ISO 639-1.
lang = 'pt'

#### Capturando os dados do twitter:
** (Já evitando mensagens repetidas) **

In [5]:
# #Cria um objeto para a captura
# api = tweepy.API(auth)

# #Inicia a captura, para mais detalhes: ver a documentação do tweepy
# i = 1
# msgs = []
# for msg in tweepy.Cursor(api.search, q=produto, lang=lang, tweet_mode="extended").items():    
#     # Evitando mensagens repitidas:
#     if msg not in msgs: 
#         msgs.append(msg.full_text.lower())
#         i += 1
#     if i > n:
#         break

# #Embaralhando as mensagens para reduzir um possível viés
# shuffle(msgs)

Salvando os dados em uma planilha Excel:

In [6]:
# #Verifica se o arquivo não existe para não substituir um conjunto pronto
# if not os.path.isfile('./{0}.xlsx'.format(produto)):
    
#     #Abre o arquivo para escrita
#     writer = pd.ExcelWriter('{0}.xlsx'.format(produto))

#     #divide o conjunto de mensagens em duas planilhas
#     dft = pd.DataFrame({'Treinamento' : pd.Series(msgs[:t])})
#     dft.to_excel(excel_writer = writer, sheet_name = 'Treinamento', index = False)

#     dfc = pd.DataFrame({'Teste' : pd.Series(msgs[t:])})
#     dfc.to_excel(excel_writer = writer, sheet_name = 'Teste', index = False)

#     #fecha o arquivo
#     writer.save()

___
### Classificando as mensagens na coragem

Esta etapa é manual. Faça a mesma pelo Excel.

#### Limpando o DataFrame

In [7]:
def deletar_caracteres(coluna):
    coluna = coluna.replace("https://","").replace("."," ").replace(":"," ").replace(","," ").replace("'"," ")\
    .replace('"', " ").replace("#","").replace("-","").replace("•", "").replace("\n\n", " ").replace("\n", " ")\
    .replace("—", " ").replace("rt", "").replace("?", " ").replace("!", " ").replace("/","").replace(";", " ")\
    .replace("{", "").replace("}", "").replace("|","").replace("(", "").replace(")", "").replace("_", "")\
    .replace("__", "").replace("\\","").replace("´", "").replace("ˆ","").replace("˜","").replace("\\", " \\")\
    .replace("U+", " U+")
    return coluna

In [8]:
def separa_emoji(em):
    em_split_emoji = emoji.get_emoji_regexp().split(em)
    em_split_whitespace = [substr.split() for substr in em_split_emoji]
    em_split = functools.reduce(operator.concat, em_split_whitespace)
    return em_split


In [9]:
Treinamento = pd.read_excel(produto + ".xlsx", sheet_name= "Treinamento")
Teste = pd.read_excel(produto +".xlsx", sheet_name= "Teste")

In [10]:
Treinamento['Treinamento'] = Treinamento['Treinamento'].apply(deletar_caracteres)
Teste["Teste"] = Teste["Teste"].apply(deletar_caracteres)

In [11]:
relevante = Treinamento[Treinamento["Classificação"]==1]
tx_relevante = " ".join(relevante.Treinamento)
tx_relevante = tx_relevante.lower()

In [12]:
irrelevante= Treinamento[Treinamento["Classificação"]==2]
tx_irrelevante = " ".join(irrelevante.Treinamento)
tx_irrelevante = tx_irrelevante.lower()

___
### Montando o Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

**O produto escolhido foi a marca Tesla Motors, avaliamos os tweets em 2 categorias (RELEVANTE e IRRELEVANTE). O critério usado para a classificação foi relativamente simples, uma vez que "Tesla" pode se referir a outras coisas como o inventor famoso Nikola Tesla, que não tem relação alguma com a marca Tesla. 
<br>
Por exemplo:**
<br>
- *"rt @anlotulys: "se quiseres compreender o segredo do universo, pense em energia, frequência e vibração." tesla"*
<br> 

Esse tweet foi marcado como IRRELEVANTE, pois se refere ao Nikola Tesla e não a marca Tesla.
<br>
**Já o tweet:**
<br>
- *"o preço de tesla coloca ele como um carro de luxo."*
<br> 

Foi marcado como RELEVANTE, pois se refere a marca Tesla

In [13]:
serie_relevante = pd.Series(separa_emoji(tx_relevante))
# serie_relevante

In [14]:
tabela_relevante = serie_relevante.value_counts()
d_lista_relev= tabela_relevante[1]
# tabela_relevante

In [15]:
tabela_relevante_relativa = serie_relevante.value_counts(True)
# tabela_relevante_relativa

In [16]:
serie_irrelevante = pd.Series(separa_emoji(tx_irrelevante))
#serie_irrelevante

In [17]:
tabela_irrelevante = serie_irrelevante.value_counts()
d_lista_irrelev= tabela_irrelevante[1]
###tabela_irrelevante

In [18]:
tabela_irrelevante_relativa = serie_irrelevante.value_counts(True)
# tabela_irrelevante_relativa

In [19]:
def probabilidade(palavra): 
    if palavra in tabela_relevante:
        k = tabela_relevante[palavra]
    else:
        k = 0
    if palavra in tabela_irrelevante:
        J = tabela_irrelevante[palavra]
    else:
        J = 0
    prob_relev = (k +1)/(len(serie_relevante) + d_lista_relev)
    prob_irrelev = (J + 1)/(len(serie_irrelevante) + d_lista_irrelev) 
    return [prob_relev, prob_irrelev]

In [20]:
def prob_frase(frase):
    f = str(frase.lower())
    f = separa_emoji(f)
    coeficiente_relevante = 1
    coeficiente_irrelevante = 1
    for e in f:
        coeficiente_relevante *= probabilidade(e)[0]
        coeficiente_irrelevante *= probabilidade(e)[1]
        
    if coeficiente_relevante >= coeficiente_irrelevante:
        return 1
    else:
        return 2
    
# print (prob_frase("se quiseres compreender o segredo do universo, pense em energia, frequência e vibração. tesla"))

In [21]:
k = []
for e in Teste["Teste"]:
    k.append(prob_frase(e))
Teste["Simulação 1"] = k   

___
### Verificando a performance

Agora você deve testar o seu classificador com a base de Testes.

In [22]:
pos = 0
falso_pos = 0
neg = 0
falso_neg = 0
total = len(Teste["Classificação"])
irrel = 0
rel = 0

for [e,i] in zip(Teste["Classificação"], Teste["Simulação 1"]):
    if e == 1:
        if i == 1:
            pos += 1
        else: 
            falso_neg += 1
    if e == 2:            
        if i == 2:
            neg +=1
        else:
            falso_pos +=1        
            
for e in Teste["Classificação"]:
    if e==1:
        rel +=1
    else:
        irrel +=1            
            

In [23]:
print("Verdadeiros relevantes: ", round((pos/total)*100,2), "%")
print("Falsos relevantes: ", round((falso_pos/total)*100, 2), "%")
print("Verdadeiros irrelevantes: ", round((neg/total)*100,2), "%")
print(f"Falsos irrelevantes: ", round((falso_neg/total)*100, 2), "%")
print("_"*55)

print("Acertos: ", round(((pos/total)+(neg/total))*100,2), "%")
print("Erros: ", round(((falso_pos/total)+(falso_neg/total))*100,2), "%")
print("_"*55)


print("Tweets classificados manualmente como relevantes:", 100*rel/len(Teste["Teste"]), "%")
print("Tweets classificados manualmente como irrelevantes:", 100*irrel/len(Teste["Teste"]), "%")

Verdadeiros relevantes:  37.0 %
Falsos relevantes:  8.5 %
Verdadeiros irrelevantes:  36.5 %
Falsos irrelevantes:  18.0 %
_______________________________________________________
Acertos:  73.5 %
Erros:  26.5 %
_______________________________________________________
Tweets classificados manualmente como relevantes: 55.0 %
Tweets classificados manualmente como irrelevantes: 45.0 %


### Fazendo o código rodar a cada hora:
tirar os "#" para habilitar

In [24]:
# n = 300
# simulacao = 1
# while True:
#     simulacao +=1
#     #Cria um objeto para a captura
#     api = tweepy.API(auth)

#     #Inicia a captura, para mais detalhes: ver a documentação do tweepy
#     i = 1
#     msgs = []
#     for msg in tweepy.Cursor(api.search, q=produto, lang=lang, tweet_mode="extended").items():    
#         # Evitando mensagens repitidas:
#         if msg not in msgs: 
#             msgs.append(msg.full_text.lower())
#             i += 1
#         if i > n:
#             break
#     dft = pd.DataFrame({'Teste' : pd.Series(msgs)})
#     dft["Teste"] = Teste["Teste"].apply(deletar_caracteres)
#     dft.dropna(inplace=True)
#     k = []
#     for e in dft["Teste"]:
#         k.append(prob_frase(e))
#     Teste["Simulação {0}".format(simulacao)] = k 

#     irrel = 0
#     rel = 0
#     for e in Teste["Classificação"]:
#         if e==1:
#             rel +=1
#         else:
#             irrel +=1
# #     print(f"relevantes: {round(rel/(rel+irrel),2)} irrelevantes: {round(irrel/(rel+irrel),2)}")
#     print(Teste.head())
#     sleep(60*15) #15 minutos

___
### Concluindo

In [25]:
print("Verdadeiros relevantes: ", round((pos/total)*100,2), "%")
print("Falsos relevantes: ", round((falso_pos/total)*100, 2), "%")
print("Verdadeiros irrelevantes: ", round((neg/total)*100,2), "%")
print(f"Falsos irrelevantes: ", round((falso_neg/total)*100, 2), "%")
print("_"*55)

print("Acertos: ", round(((pos/total)+(neg/total))*100,2), "%")
print("Erros: ", round(((falso_pos/total)+(falso_neg/total))*100,2), "%")
print("_"*55)


print("Tweets classificados manualmente como relevantes:", 100*rel/len(Teste["Teste"]), "%")
print("Tweets classificados manualmente como irrelevantes:", 100*irrel/len(Teste["Teste"]), "%")

Verdadeiros relevantes:  37.0 %
Falsos relevantes:  8.5 %
Verdadeiros irrelevantes:  36.5 %
Falsos irrelevantes:  18.0 %
_______________________________________________________
Acertos:  73.5 %
Erros:  26.5 %
_______________________________________________________
Tweets classificados manualmente como relevantes: 55.0 %
Tweets classificados manualmente como irrelevantes: 45.0 %


    Como impresso novamente na célula a cima, notamos uma taxa que julgamos boa de acertos por parte de nosso algorítimo, de 73.5%, e analisando por grupo de relevância, temos proporções muito próximas entre relevantes e irrelevantes nas avaliações feitas manualmente e pelo Naive-Bayes (55% de relevantes na avaliação manual e 45% pelo Naive-Bayes) concluindo não haver nenhuma tendência sistêmica de direcionamento por parte do algorítmo.
    Uma falha encontrada no desenvolver do projeto foi ao implementarmos uma função que separava os "emojis" e os considerava cada um uma palavra, a taxa de acerto do código diminuiu em pouco menos de 5%, possivalmente em virtude de uma maior repetição de uma palavra (normalmente emojis são usados em sequencia entao passam a ter uma frequencia muito maior com a nova função) influencia diretamente no cálculo do Naive-Bayes en suas próximas ocorrências.
    Como modo de ampliação e aprofundamento do projeto deveria ser implementado um sistema capaz de interpreetar duplas negativas, sinais de ironia e principalmente entender "erros de ortografia" e variações à norma padrão, cabendo também o acrescimo de mais categorias dentro do grupo "relevante" de forma a dicernir reais preferências do usuário à respeito do produto analizado.
    
    


## Aperfeiçoamento:

Os trabalhos vão evoluir em conceito dependendo da quantidade de itens avançados:

* Limpar: \n, :, ", ', (, ), etc SEM remover emojis
* Corrigir separação de espaços entre palavras e emojis ou emojis e emojis
* Propor outras limpezas e transformações que não afetem a qualidade da informação ou classificação
* Criar categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante (3 categorias: C, mais categorias conta para B)
* Explicar por que não posso usar o próprio classificador para gerar mais amostras de treinamento
* Propor diferentes cenários para Naïve Bayes fora do contexto do projeto
* Sugerir e explicar melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)
* Montar um dashboard que periodicamente realiza análise de sentimento e visualiza estes dados

#### - Explicar por que não posso usar o próprio classificador para gerar mais amostras de treinamento:

Não podemos fazer isso por causa da propagação de erro, se temos 70% de acurácia ao usarmos esse mesmo classificador para aumentar a base de dados significará que apenas 70% das mensagens novas estarão classificadas corretamente, e portanto o classificador treinará com mensagens classificadas equivocadamente e diminuirá a acurácia do classificador.

#### - Propor diferentes cenários para Naïve Bayes fora do contexto do projeto:

Podemos usar o Naive Bayes para sistemas de reconhecimento de imagem, por exemplo fazer um programa que reconhece números e letras escritos a mão em imagens. Para fazer isso usamos o os pixels das imagens em "grayscale" com o vetor posição de cada um. 

$$
P(c|x) = \frac{P(x | c)P(c)}{P(x)}
$$

Onde "x" representa a informação dada e "c" a classe que a representa, no caso "x" seria o pixel vetorizado e "c" o caracter.
<br>
Porém para calcular o $ P(c)$ não temos como somar todos os pixels de todas as imagens, por isso usamos a forma normal multivariada em escala logarítimica.
 



# Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**

link para separação dos Emojis:
<br>
https://stackoverflow.com/questions/49921720/how-to-split-emoji-from-each-other-python    

link para remoção de caracteres:
<br>
https://pt.stackoverflow.com/questions/217832/como-retirar-caractere-especial-e-ponto-de-coluna-string-de-um-data-frame

outras aplicações do Naive Bayes:
<br>
https://www.quora.com/In-what-real-world-applications-is-Naive-Bayes-classifier-used
<br>
https://lazyprogrammer.me/bayes-classifier-and-naive-bayes-tutorial-using/
